# Station configuration

## Setting up a station manually

In QCoDeS you can quickly set up your experiment using stations and YAML files. In this tutorial you will learn how, but before we come to this, you will learn what a station is, why you should use it and how to define a it the good old manual way. This goes by first importing the necessary components and defining your parameters:

In [1]:
import qcodes as qc
from qcodes import Station
from qcodes.tests.instrument_mocks import DummyInstrument
from qcodes.instrument.parameter import Parameter
from qcodes.instrument.specialized_parameters import ElapsedTimeParameter

src1 = DummyInstrument(name='src1', gates=['v1', 'v2'])
src2 = DummyInstrument(name='src2', gates=['v3', 'v4'])

t = ElapsedTimeParameter('t')
V_appl_1 = Parameter('V_appl1', label='V_appl', unit='V', get_cmd=src1.v1.get, set_cmd=src1.v1.set)
V_appl_2 = Parameter('V_appl2', label='V_appl', unit='V', get_cmd=src2.v3.get, set_cmd=src2.v4.set)
I_meas = Parameter('I', label='I_meas', unit='A', get_cmd=lambda: float((src1.v1()+src2.v3())/1000))

Next we create a station and import the the parameters and instruments into it. This way we can keep track of all our experimental parameters and the instruments we use in the particular experiment. This keeps the workspace clean and will make the understand of the dataset later on much easier. In this example we add an instrument and a parameter and later on confrim that the components are contained in the station:

In [2]:
station = Station(I_meas)#Initializing a station with the I_meas parameter
print('Station after initialization: ')
print(station.components)
station.add_component(t) #example of adding a parameter
station.add_component(src1) #exmaple of adding an instrument
print('Added components to station: ')
print(station.components)
station.remove_component('I') #example of adding a parameter
print('Removed the I_meas component from the station: ')
print(station.components)

Station after initialization: 
{'I': <qcodes.instrument.parameter.Parameter: I at 4715995664>}
Added components to station: 
{'I': <qcodes.instrument.parameter.Parameter: I at 4715995664>, 't': <qcodes.instrument.specialized_parameters.ElapsedTimeParameter: t at 4715965072>, 'src1': <DummyInstrument: src1>}
Removed the I_meas component from the station: 
{'t': <qcodes.instrument.specialized_parameters.ElapsedTimeParameter: t at 4715965072>, 'src1': <DummyInstrument: src1>}


This enables us now to access the parameters via the station, which will come in handy later on, when we define all the parameters via the YAML file. Here is an example of how to read the elaspsed time parameter via the station:

In [3]:
station.t()

0.289901545

## Creating snapshots of a station

In order to store all the relevant metadata for your experiment the snapshot function is used. The snapshot produces a json type dictionary file wich can be navigate through like a python dictionary. Here is an example of how easy it is to create a snapshot:

In [4]:
station.snapshot()

{'instruments': {'src1': {'functions': {},
   'submodules': {},
   '__class__': 'qcodes.tests.instrument_mocks.DummyInstrument',
   'parameters': {'IDN': {'value': {'vendor': None,
      'model': 'src1',
      'serial': None,
      'firmware': None},
     'raw_value': {'vendor': None,
      'model': 'src1',
      'serial': None,
      'firmware': None},
     'ts': '2020-06-23 15:13:13',
     '__class__': 'qcodes.instrument.parameter.Parameter',
     'full_name': 'src1_IDN',
     'instrument': 'qcodes.tests.instrument_mocks.DummyInstrument',
     'instrument_name': 'src1',
     'vals': '<Anything>',
     'label': 'IDN',
     'post_delay': 0,
     'name': 'IDN',
     'inter_delay': 0,
     'unit': ''},
    'v1': {'value': 0,
     'raw_value': 0,
     'ts': '2020-06-23 15:13:13',
     '__class__': 'qcodes.instrument.parameter.Parameter',
     'full_name': 'src1_v1',
     'instrument': 'qcodes.tests.instrument_mocks.DummyInstrument',
     'instrument_name': 'src1',
     'vals': '<Numbers -

Since this is pretty long and not usefull for a quick check before the measurement (or during a measurement), there is a function that produces a readble shnapshot:

In [5]:
station.src1.print_readable_snapshot()

src1:
	parameter value
--------------------------------------------------------------------------------
IDN :	{'vendor': None, 'model': 'src1', 'serial': None, 'firmware': None} 
v1  :	0 (V)
v2  :	0 (V)


# YAML file station configuration

Now we have to transform this into a YAML file and then initialize the station by such a file. But the YAML file is more than just a helper for intialization. You can also use it in order to extend the parameter set, change labels and set limits. Here is just a brief example for an initilaization which we will run in just a second, but there is more in the official QCoDeS documentation:

## Example YAML file

Here is an example of what a YAML file's content looks like:
```
instruments: 
    src3: 
        type: qcodes.tests.instrument_mocks.DummyInstrument
        init:
            gates: {v5, v6}
        enable_forced_reconnect: true
        parameters:
            v5:
                label: Gate 5
                unit: V
                initial_value: 0
                monitor: true
            v6: {label: Gate 6 ,unit: V ,initial_value: 0,monitor: true}

```

This file is stored as init.yaml in the working directory and in the next step we will use this in order to initialize a new station:

## Initializing a station from YAML

It is as easy as that:

In [6]:
station = Station(config_file='init.yaml')

How do we get the instrument?

In [7]:
loaded_src3 = station.load_instrument('src3')
#or instead just
loaded_src3 = station.load_src3()

The loaded instrument has been initialized with v5=0V, we can check this by:

In [8]:
loaded_src3.v5()

0

Note: It is important that this initialization comes with a bit of a caveat, which is that the real instruments have to be closed later on:

In [9]:
loaded_src3.close_and_remove_instrument()

AttributeError: 'DummyInstrument' object and its delegates have no attribute 'close_and_remove_instrument'

Having a close look at this YAML file from above, we see that it has some similarity with the snapshot we printed above, so the question arises, if it is possible to create such a YAML file from an existing snapshot.

## Creating YAML a bit easier

YAML in fact is a superset of JSON: How to turn JSON into YAML? Go and visit https://www.json2yaml.com

YAML extensions by RedHat https://marketplace.visualstudio.com/items?itemName=redhat.vscode-yaml

## Store the snapshot of the station in the database

The metadata from the snapshot asociated with the station can simply be stored with the data by initializing the Measurement object for the database with the station. Here is how you save it next to the measurements data (this happens at the beginning of the measurement):

In [10]:
from qcodes import initialise_database, load_or_create_experiment
from qcodes.dataset.measurements import Measurement

exp_name = 'DC-IV-Sweep'
sample_name = 'Test-Resistor'

qc.config["core"]["db_location"]='./experiments.db'
initialise_database()
exp = load_or_create_experiment(exp_name,sample_name)

measurement = Measurement(exp, station)

measurement.register_parameter(loaded_src3.v5)
measurement.register_parameter(loaded_src3.v6,setpoints=[loaded_src3.v5])

Let's run the experiment and feed the database:

In [11]:
with measurement.run() as data_saver:
    for i in range(10):
        input_value = i
        loaded_src3.v5.set(input_value)

        loaded_src3.v6(i*100e-3)  # assuming that the instrument measured this value on the output

        data_saver.add_result((loaded_src3.v5, input_value),
                              (loaded_src3.v6, loaded_src3.v6()))

# For convenience, let's work with the dataset object directly
dataset = data_saver.dataset

Starting experimental run with id: 9. 


Now we can open the snapshot that is stored with the dataset by using:

In [12]:
dataset.snapshot

{'station': {'instruments': {'src3': {'functions': {},
    'submodules': {},
    '__class__': 'qcodes.tests.instrument_mocks.DummyInstrument',
    'parameters': {'IDN': {'value': {'vendor': None,
       'model': 'src3',
       'serial': None,
       'firmware': None},
      'raw_value': {'vendor': None,
       'model': 'src3',
       'serial': None,
       'firmware': None},
      'ts': '2020-06-23 15:13:16',
      '__class__': 'qcodes.instrument.parameter.Parameter',
      'full_name': 'src3_IDN',
      'instrument': 'qcodes.tests.instrument_mocks.DummyInstrument',
      'instrument_name': 'src3',
      'vals': '<Anything>',
      'label': 'IDN',
      'post_delay': 0,
      'name': 'IDN',
      'inter_delay': 0,
      'unit': ''},
     'v5': {'value': 0,
      'raw_value': 0,
      'ts': '2020-06-23 15:13:17',
      '__class__': 'qcodes.instrument.parameter.Parameter',
      'full_name': 'src3_v5',
      'instrument': 'qcodes.tests.instrument_mocks.DummyInstrument',
      'instrument

## Comparing two different Snapshots

Sometimes you have run several seemlingly identical measurements and you forgot what was the exact difference between them and want to look it up quickly. This is where this quick function comes in handy:

In [13]:
from qcodes.utils.metadata import diff_param_values
from qcodes import load_by_id

snapshot1 = dataset.snapshot
snapshot2 = load_by_id(8).snapshot # in Run 8 I changed the v5 init voltage to 1

diff_param_values(snapshot1, snapshot2).changed

{('src3', 'v5'): (0, 1)}

## Sources

Documentation: "Station" (https://qcodes.github.io/Qcodes/examples/Station.html)


Documentation: "Working with Snapshots" (https://qcodes.github.io/Qcodes/examples/DataSet/Working%20with%20snapshots.html)